# SARSA Trading Algorithm

This notebook implements a way to evaluate S&P 500 index

In [10]:
import pandas as pd
import numpy as np

## Data Loading and Processing

In [11]:
df = pd.read_csv("/home/youssouf/Downloads/archive/sp500_5yr.csv")

In [12]:
# Compute the index prices (volume-weighted average)
unique_symbols = df['Name'].unique()
new_df = {}
dates = None

for i, symbol in enumerate(unique_symbols):
    partial_df = df[df['Name'] == symbol].copy()
    partial_df = partial_df.sort_values(by='date')

    if partial_df.isna().sum().sum() > 0:
        continue
    if dates is None:
        dates = partial_df['date'].to_list()
        if len(dates) < 1259: 
            continue
    elif len(partial_df) < 1259:
        continue
    else:
        assert dates == partial_df['date'].to_list(), "Dates do not match across symbols"
    
    for col in ['open', 'high', 'low', 'close', 'volume']:
        new_df[col + f"-{symbol}"] = partial_df[col].to_list()

new_df = pd.DataFrame(new_df)

# Convert to float64
for col in new_df.columns:
    new_df[col] = new_df[col].astype(np.float64)

new_df['date'] = dates
new_df = new_df.set_index('date')

print(f"Processed data shape: {new_df.shape}")
print(f"Date range: {new_df.index[0]} to {new_df.index[-1]}")

Processed data shape: (1259, 2340)
Date range: 2013-02-08 to 2018-02-07


In [13]:
# Compute volume-weighted index prices
index_prices = pd.DataFrame()
volume_cols = new_df.filter(like='volume-')
total_volume = volume_cols.sum(axis=1)

for price_type in ['open', 'high', 'low', 'close']:
    price_cols = new_df.filter(like=f'{price_type}-')
    index_prices[price_type] = (price_cols.values * volume_cols.values).sum(axis=1) / total_volume.values

index_prices.index = new_df.index
print(f"Index prices shape: {index_prices.shape}")
index_prices.head()

Index prices shape: (1259, 4)


,open,high,low,close
date,,,,
2013-02-08,41.082020,41.500640,40.789177,41.207748
2013-02-11,40.832917,41.173278,40.513390,40.862581
2013-02-12,39.197571,39.582484,38.799817,39.161757
2013-02-13,38.675202,39.027626,38.276385,38.630306
2013-02-14,39.466134,40.028357,39.084703,39.634254


## Save Data

In [14]:
index_prices.to_csv('sp500_index_prices.csv')

In [15]:
index_prices.isna().sum().sum()  # Check for NaN values

np.int64(0)